<a href="https://colab.research.google.com/github/tenoriolms/databank_CH4/blob/main/_module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://docs.python.org/pt-br/3/tutorial/modules.html

In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import missingno as msno


import sklearn.metrics

In [ ]:
'''
import sklearn.datasets
import sklearn.model_selection

import sklearn.ensemble #bibliotecas de aprendizado de máquina
'''

'\nimport sklearn.datasets\nimport sklearn.model_selection\n\nimport sklearn.ensemble #bibliotecas de aprendizado de máquina\n'

#Declaração de Variáveis

In [ ]:
data = '962' #ID do databank
git_import_file = '' #nome da variante do databank original
data_url = ''


#dict para armazenar dados da função ID
ID_dict = {}

#gases com permeabilidades presentes no banco de dados ordem crescente de kinetic diameter
#EM ORDEM CRESCENTE AO D.C., COMO NO DATABANK:
gases = ('He','H2','CO2','O2','H2S','N2','CO','CH4','C2H6','SF6') #referentes aos dados de permeabilidade
gases_kinetic_diameter = {'He':260,
                          'H2':289,
                          'CO2':330,
                          'Ar':340,
                          'O2':346,
                          'H2S':360,
                          'N2':364,
                          'CO':376,
                          'CH4':380,
                          'C2H6':444.3,
                          'SF6':550,
                          'none':0}
gases_effec_diameter = {'He':178, #T>Tg
                        'H2':214,
                        'CO2':302,
                        'O2':289,
                        'Ar':297,
                        'H2S':np.nan,
                        'N2':304,
                        'CO':304,
                        'CH4':318,
                        'C2H6':346,
                        'SF6':np.nan,
                        'none':0}
gases_molar_mass = {'He':4.00, 
                    'H2':2.02,
                    'CO2':44.01,
                    'O2':31.98,
                    'Ar':39.95,
                    'H2S':34.10,
                    'N2':28.00,
                    'CO':28.01,
                    'CH4':16.04,
                    'C2H6':30.07,
                    'SF6':146.06,
                    'none':0.}
gases_polarizability = {'He':0.208, #https://cccbdb.nist.gov/pollistx.asp
                        'H2':0.787,
                        'CO2':2.507,
                        'O2':1.562,
                        'Ar':1.664,
                        'H2S':3.631,
                        'N2':1.710,
                        'CO':1.953,
                        'CH4':2.448,
                        'C2H6':4.226,
                        'SF6':4.490,
                        'none':0.}


gases_kinetic_diameter_inverse = {}
for i in gases_kinetic_diameter.keys():
  gases_kinetic_diameter_inverse[ gases_kinetic_diameter[i] ] = i

gases_effec_diameter_inverse = {}
for i in gases_effec_diameter.keys():
  gases_effec_diameter_inverse[ gases_effec_diameter[i] ] = i
                        
#frequencia de dados para cada gas em situação "pura" e "mixtura"
count_pure = {}
count_mixture = {}


#nome das colunas correspondentes à características da membrana e do processo e 
#seus respectivos índices na coluna
#EM ORDEM:
columns_membrane = ['type', 'description', 'support_material', 'subtype', 'filler_loading',
                    'previous_thickness', 'mean_thickness', 'mean_pore_size', 'pore_volume',
                    'specific_surface_area',  'aging']
columns_process = ['surface_area', 'temperature', 'feed_pressure', 'permeate_pressure',
                   'delta_pressure', 'feed_flow_rate', 'sweep_gas', 'sweep_gas_flow',
                   'stage_cut']
columns_others = ['provided_data_type', 'in_reference_data_location', 'reference', 'url']

columns_membrane_index = {}
columns_process_index = {}
columns_others_index = {}

#A performance de cada gás foi representada pela seguintes variáveis:
prefix1='x_' #Fração mássica/molar/volumétrica
prefix2='Py_' #Permeabilidade
prefix3='Pe_' #Permeância

#VARIÁVEIS AUXILIARES:
dados = pd.DataFrame() #Dataframe a ser utilizados para a previsão. Versão refinada.

##GitHub info

In [ ]:
git_username = 'tenoriolms' #username no GitHub
git_repository = 'databank_CH4' #Nome do repositório

git_token = 'não pode' #token para acesso do repositório.
#O github possui um algoritmo para verificar se dentro de cada arquivo importado/commitado
#existe o token de acesso criado, que é secreto. Caso existir, esse token é revogado.
#Como esse notebook irá ser exportado para o github. O token não pode ser escrito aqui.

!git config --global user.email "lhucas_tenorio@hotmail.com"
!git config --global user.name "tenoriolms"

#Funções

##ID(index)

In [ ]:
#ID = string utilizada para identificar uma membrana e suas circunstâncias de utilização no banco de dados
#A ID é a soma das variáveis (em forma de strings) que podem variar em uma dada referência
def ID(i, df):
  if i in ID_dict:
    return ID_dict[i]
  else:
    
    aux = str(df['description'][i]) + str(df['filler_loading'][i]) \
    + str(df['mean_thickness'][i]) + str(df['mean_pore_size'][i]) \
    + str(df['temperature'][i]) + str(df['feed_pressure'][i]) \
    + str(df['delta_pressure'][i]) + str(df['feed_flow_rate'][i]) \
    + str(df['stage_cut'][i]) + str(df['aging'][i]) \
    + str(df['reference'][i])
    ID_dict[i] = aux
    
    return aux

##get_key(val,my_dict))

In [ ]:
def get_key(val,my_dict): #dict = key : value
    for key, value in my_dict.items():
         if val == value:
             return key
 
    return "get_key function: There is no such Key"

##submit_file(git_export_file)

In [ ]:
#exportar para o GitHub
def submit_file(git_export_file):
  !git clone https://{git_token}@github.com/{git_username}/{git_repository}
  !cp {git_export_file} {git_repository}
  %cd {git_repository}
  !git add {git_export_file}
  !git commit -m 'Add/Atualizar arquivo {input_file}'
  !git push -u origin
  %cd ..
  !rm -rf {git_repository}

##import_file(git_import_file)

In [ ]:
#importar um arquivo do GitHub
def import_file(git_import_file):
  !git clone https://{git_token}@github.com/{git_username}/{git_repository}
  !cp {git_repository}/{git_import_file} .
  !rm -rf {git_repository}

##Zscores(df_for_scaled, df_reference)

In [ ]:
#Escalonar cada coluna utilizando o "z score"
def Zscores(df_for_scaled, df_reference):
  for i in df_for_scaled.columns: 
    df_for_scaled[i] = (df_for_scaled[i] - df_reference[i].mean()) / df_reference[i].std()     
  #return df_for_scaled

##undo_Zscores(df_scaled, df_reference)

In [ ]:
#desfazer o escalonamento realizado para cada coluna utilizando o "z score"
def undo_Zscores(df_scaled, df_reference):
  for i in df_scaled.columns:
    df_scaled[i] = df_scaled[i]*df_reference[i].std() + df_reference[i].mean()
  #return df_scaled

##str2int_simple_encoder(df,columns='all')

In [ ]:
def str2int_simple_encoder(df,columns='all'):
  import pandas as pd
  
  id_dict = {}
  if (columns=='all'):
    
    for i in df.columns:
      if (df[i].dtype==object):
        id_dict[i] = {}
        unique_values = df[i].unique()
        id_dict[i] = {name: id + 1 for id, name in enumerate(unique_values)}

        df[i] = df[i].apply(lambda row, value : value[row], value = id_dict[i] )

  else:
    
    for i in columns:
      if ( (df[i].dtype==object) and (i in df.columns) ):
        id_dict[i] = {}
        unique_values = df[i].unique()
        id_dict[i] = {name: id + 1 for id, name in enumerate(unique_values)}

        df[i] = df[i].apply(lambda row, value : value[row], value = id_dict[i] )
      else:
        print('coluna especificada não é do tipo "object" ou não existe no dataframe')
        return
  
  return id_dict

In [ ]:
#como era feito anteriormente:
'''
df = dados

#Criar os dicionários para os valores únicos das colunas categóricas
type_id = {}
aux = df['type'].unique()
for i in aux:
  type_id[i] = np.where(aux==i)[0][0]+1
print(type_id)

#converter os valores categóricos da coluna "type" por numéricos
df['type'] = df['type'].apply(lambda row, value : value[row],
                                                    value = type_id )

'''

'\ndf = dados\n\n#Criar os dicionários para os valores únicos das colunas categóricas\ntype_id = {}\naux = df[\'type\'].unique()\nfor i in aux:\n  type_id[i] = np.where(aux==i)[0][0]+1\nprint(type_id)\n\n#converter os valores categóricos da coluna "type" por numéricos\ndf[\'type\'] = df[\'type\'].apply(lambda row, value : value[row],\n                                                    value = type_id )\n\n'

##display_score(m,x_train,x_test,y_train,y_test))

Out-of-bag parameter:

https://towardsdatascience.com/what-is-out-of-bag-oob-score-in-random-forest-a7fa23d710

https://www.analyticsvidhya.com/blog/2020/12/out-of-bag-oob-score-in-the-random-forest-algorithm/

https://stats.stackexchange.com/questions/88980/why-on-average-does-each-bootstrap-sample-contain-roughly-two-thirds-of-observat

https://stats.stackexchange.com/questions/198839/evaluate-random-forest-oob-vs-cv

In [ ]:
def rmse(v_real,v_pred): 
    return np.sqrt(sklearn.metrics.mean_squared_error(v_real,v_pred)) #leia sobre sklearn.metrics.mean_squared_error
def r2(v_real,v_pred): 
    return sklearn.metrics.r2_score(v_real,v_pred) #leia sobre sklearn.metrics.r2_score

##função para avaliar RMSE, R2 e OOB_score
def display_score(m,x_train,x_test,y_train,y_test):
    
    res = [[rmse( y_train,m.predict(x_train) ), r2( y_train,m.predict(x_train) )],
          [rmse( y_test,m.predict(x_test) ), r2( y_test,m.predict(x_test) )]] #a função display score irá retornar uma tabela
    
    score = pd.DataFrame(res, columns=['RMSE','R2'], index = ['Treino','Teste'])

    if hasattr(m, 'oob_score_'): #https://www.programiz.com/python-programming/methods/built-in/hasattr
        score.loc['OOB'] = [rmse(y_train, m.oob_prediction_), m.oob_score_]

    display(score)

##Gráficos

### plt_hist_by(df,variable,by)

In [ ]:
def plt_hist_by(df = pd.DataFrame(),
                variable = [],
                by = '',
                consider_none = False,
                consider_zeros = False):
  #https://plotly.com/python/histograms/
  import plotly.graph_objects as go

  fig = go.Figure()

  #Filtrar apenas as linha que possuem dados de "variavel"
  for i in variable:
    
    if (consider_none==False):
      df.loc[ df[i]=='none', i ] = np.nan
      df.loc[ df[i]=='None', i ] = np.nan

    if (consider_zeros==False):
      df.loc[ df[i]==0, i ] = np.nan

    df_aux = df.loc[ df[i].notna(), by]

    fig.add_trace(go.Histogram(
        x=df_aux,
        histnorm='',
        name=i, # name used in legend and hover labels
        #marker_color='#EB89B5',
        #opacity=0.75
        ))

  fig.update_layout(
      title_text=f'Quantity of data by each {by}', # title of plot
      xaxis_title_text=by, # xaxis label
      yaxis_title_text='Count', # yaxis label
      bargap=0.2, # gap between bars of adjacent location coordinates
      bargroupgap=0.1 # gap between bars of the same location coordinates
      )

  fig.show()

### plt_hist_columns(df)

In [ ]:
def plt_hist_columns(df):
  aux = []
  for i in df.columns:
    aux += [i]
    if (len(aux)==4):
      try:
        df[aux] = df[aux].astype(float)
      except:
        print()
      df[aux].hist()
      aux = []
  df[aux].hist()

### heatmap_corr(df, x='all', y='all')

In [ ]:
def heatmap_corr(df, x='all', y='all', method='pearson', min_periods=1, color='di'):
  import pandas as pd
  
  corr_pear = df.corr( min_periods=min_periods, method=method )
  
  if (x=='all'):
    x = corr_pear.columns.tolist()
  #print(x)
  if (y=='all'):
    y = corr_pear.columns.tolist()
  #print(y)
  heatmap_pearson = pd.DataFrame( columns=x, index=y )
  heatmap_pearson = corr_pear.loc[y,x]
  
    #GRAFICO#
  f, ax = plt.subplots(figsize=( 1*len(x)+3, 1*len(y) ))
  if color=='mono':
    colors = ('#00076e', '#1b00ff', '#d0cbff', '#FFFFFF', '#d0cbff', '#1b00ff', '#00076e')
  elif (color=='di'):
    colors = ('#7e0000', '#ff0000', '#fecfcf', '#FFFFFF', '#d0cbff', '#1b00ff', '#00076e')
  cmap = sns.blend_palette(colors, input='rgb', as_cmap=True)
  sns.heatmap(heatmap_pearson, annot=True, cmap=cmap, ax=ax, center=0) 

  return heatmap_pearson

###draw_tree(t, dados, size=10, ratio=1, precision=0)

In [ ]:
def draw_tree(t, dados, size=10, ratio=1, precision=0):
   
    import re
    import graphviz
    import sklearn.tree
    import IPython.display
    
    s=sklearn.tree.export_graphviz(t, out_file=None, feature_names=dados.columns, filled=True,
                                   special_characters=True, rotate=True, precision=precision)
    IPython.display.display(graphviz.Source(re.sub('Tree {',
       f'Tree {{ size={size}; ratio={ratio}', s)))

###plotar_importancias(modelo, tags, n=10) - modelo RF

In [ ]:
def plotar_importancias(modelo, tags, n=10):
    
    fig, ax = plt.subplots(1,2, figsize = (20,4))

    coefs = []
    abs_coefs = []

    if hasattr(modelo,'coef_'):
        imp = modelo.coef_
    elif hasattr(modelo,'feature_importances_'):
        imp = modelo.feature_importances_
    else:
        print('sorry, nao vai rolar!')
        return

    coefs = (pd.Series(imp, index = tags))
    coefs.plot(use_index=False, ax=ax[0]);
    abs_coefs = (abs(coefs)/(abs(coefs).sum()))
    abs_coefs.sort_values(ascending=False).plot(use_index=False, ax=ax[1],marker='.')

    ax[0].set_title('Importâncias relativas das variáveis')
    ax[1].set_title('Importâncias relativas das variáveis - ordem decrescente')

    abs_coefs_df = pd.DataFrame(np.array(abs_coefs).T,
                                columns = ['Importancias'],
                                index = tags)

    df = abs_coefs_df['Importancias'].sort_values(ascending=False)
    
    print(df.iloc[0:n])
    plt.figure()
    df.iloc[0:n].plot(kind='barh', figsize=(15,0.25*n), legend=False)
    
    return df